# Retrieve climate data through the meteo.cat API

In [7]:
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import time
import csv
import geojson
import folium
from folium.plugins import HeatMap


In [3]:
# API key
key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1/variables/mesurades/metadades'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url)

display(r.status_code)
display(r.json())

403

{'message': 'Forbidden'}

In [39]:
# Get request to check resource availability

url = 'https://api.meteo.cat/quotes/v1/consum-actual'

headers = {'X-API-KEY': 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'}
requests.get(url, headers=headers).json()

{'client': {'nom': 'Pau Torrente Badia UB'},
 'plans': [{'nom': 'XEMA_750 OD',
   'periode': 'Mensual',
   'maxConsultes': 750,
   'consultesRestants': 99993,
   'consultesRealitzades': -99243},
  {'nom': 'Predicció_100',
   'periode': 'Mensual',
   'maxConsultes': 100,
   'consultesRestants': 100,
   'consultesRealitzades': 0},
  {'nom': 'XDDE_250',
   'periode': 'Mensual',
   'maxConsultes': 250,
   'consultesRestants': 250,
   'consultesRealitzades': 0},
  {'nom': 'Quota',
   'periode': 'Mensual',
   'maxConsultes': 300,
   'consultesRestants': 295,
   'consultesRealitzades': 5},
  {'nom': 'Referència Bàsic',
   'periode': 'Mensual',
   'maxConsultes': 2000,
   'consultesRestants': 2000,
   'consultesRealitzades': 0}]}

In [5]:
# Example of a get request retrieval

key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url, headers=headers)

r.json()

[{'codi': 'X8',
  'variables': [{'codi': 1,
    'lectures': [{'data': '2020-03-02T00:00Z',
      'dataExtrem': '2020-03-02T00:00Z',
      'valor': 995.1,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2020-03-02T00:30Z',
      'dataExtrem': '2020-03-02T00:33Z',
      'valor': 994,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2020-03-02T01:00Z',
      'dataExtrem': '2020-03-02T01:02Z',
      'valor': 993.6,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2020-03-02T01:30Z',
      'dataExtrem': '2020-03-02T01:32Z',
      'valor': 992.8,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2020-03-02T02:00Z',
      'dataExtrem': '2020-03-02T02:00Z',
      'valor': 991.6,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2020-03-02T02:30Z',
      'dataExtrem': '2020-03-02T02:30Z',
      'valor': 990.5,
      'estat': 'V',
      'baseHoraria': 'SH'},
     {'data': '2020-03-02T03:00Z',
      'dataExtrem': '2020-03-02T03:

In [7]:
#Request example with processing to retrieve rain and temperature for each timestamp and store them as dictionaries on a list

key = 'enJH8FUX2z5Ar7NSCJvYI8pAIuDW0XDV9nbSkEMj'
url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/2020/03/02'

headers = {'Accept': 'application/json', 'X-API-KEY': key}

r = requests.get(url, headers=headers)

data = r.json()[0]

# 35 is for precipitation in mm and 32 for temperature in ºC

precipitation = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]
temperature = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 32][0]


date_variables = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in precipitation]

for i in range(len(date_variables)):
    date_variables[i]['temperature'] = temperature[i]['valor']

date_variables

[{'timestamp': 1583103600, 'mm_precip': 0, 'temperature': 14},
 {'timestamp': 1583105400, 'mm_precip': 0, 'temperature': 14.9},
 {'timestamp': 1583107200, 'mm_precip': 0, 'temperature': 16},
 {'timestamp': 1583109000, 'mm_precip': 0, 'temperature': 16},
 {'timestamp': 1583110800, 'mm_precip': 0, 'temperature': 15.2},
 {'timestamp': 1583112600, 'mm_precip': 0.2, 'temperature': 14.5},
 {'timestamp': 1583114400, 'mm_precip': 0.2, 'temperature': 14.2},
 {'timestamp': 1583116200, 'mm_precip': 0, 'temperature': 14.5},
 {'timestamp': 1583118000, 'mm_precip': 0, 'temperature': 15.2},
 {'timestamp': 1583119800, 'mm_precip': 0, 'temperature': 15.6},
 {'timestamp': 1583121600, 'mm_precip': 0, 'temperature': 15.7},
 {'timestamp': 1583123400, 'mm_precip': 0, 'temperature': 15.9},
 {'timestamp': 1583125200, 'mm_precip': 0, 'temperature': 15.9},
 {'timestamp': 1583127000, 'mm_precip': 0, 'temperature': 16},
 {'timestamp': 1583128800, 'mm_precip': 0, 'temperature': 16},
 {'timestamp': 1583130600, 'mm_

In [9]:
len(date_variables) #checking that indeed our register got 48 entrances (2 per hour for a whole day)

48

In [20]:
start_date = dt.date(2019, 1, 1)
end_date = dt.date(2019, 1, 3)

assert start_date < end_date, 'Start date must be before end date'

delta = dt.timedelta(days=1)

freq = 1/20

day = start_date

columns = ['timestamp', 'mm_precip', 'temperature']

with open('weather_data/weather.csv', mode = 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()

    while day <= end_date:

        day_string = day.strftime('%Y/%m/%d')

        url = 'https://api.meteo.cat/xema/v1' + '/estacions/mesurades/X8/' + day_string 

        print(day_string)

        headers = {'Accept': 'application/json', 'X-API-KEY': key}

        data = requests.get(url, headers=headers).json()[0]

        precipitation = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 35][0]
        temperature = [data['variables'][i]['lectures'] for i in range(len(data['variables'])) if data['variables'][i]['codi'] == 32][0]


        date_variables = [{'timestamp':int(dt.datetime.strptime(d['data'], '%Y-%m-%dT%H:%MZ').timestamp()), 'mm_precip':d['valor']} for d in precipitation]

        for i in range(len(date_variables)):
            date_variables[i]['temperature'] = temperature[i]['valor']

        writer.writerows(date_variables)

        day += delta

        time.sleep(freq)



2019/01/01
2019/01/02
2019/01/03


In [43]:
# function to add the corrected timestamps as a new column for all time collmns desired to be able to merge with the weather data later

def weather_time(dataframe, columns):

    for column in columns:
        column_name = 'wt_'+column
        dataframe[column_name] = dataframe[column]- dataframe[column]%1800 + 1800
    
    return dataframe


# First trial

# gener_estacions['weather_time'] = gener_estacions['last_reported'] - gener_estacions['last_reported']%1800 + 1800

# gener_estacions.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl,weather_time
0,1,43,43,0,3,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22,1.609456e+09
1,2,17,15,2,10,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22,1.609456e+09
2,3,3,3,0,22,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22,1.609456e+09
3,4,15,14,1,6,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22,1.609456e+09
4,5,8,8,0,31,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22,1.609456e+09


# Station localization on a map using the folium library

In [14]:
import pandas as pd
import datetime as dt

gener_info = pd.read_csv('data/2021_01_Gener_BicingNou_INFORMACIO.csv')

gener_info.head()



,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,nearby_distance,cross_street,last_updated,ttl
0,1,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16,"GRAN VIA CORTS CATALANES, 760",8013,46,1000,NaN,1609455629,25
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17,"C/ ROGER DE FLOR, 126",8013,27,1000,NaN,1609455629,25
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11,"C/ NÀPOLS, 82",8013,27,1000,NaN,1609455629,25
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8,"C/ RIBES, 13",8013,21,1000,NaN,1609455629,25
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018,39,1000,NaN,1609455629,25


In [34]:
frequencies = pd.DataFrame(np.transpose(np.unique(gener_info['station_id'], return_counts=True)))

frequencies.columns = ['station_id', 'frequency']

stations = pd.DataFrame(np.unique(gener_info[['station_id','lat', 'lon']], axis = 0))

stations.columns = ['station_id', 'lat', 'lon']

frequencies  = pd.merge(left = frequencies, right = stations, on = 'station_id', how = 'left')

frequencies



,station_id,frequency,lat,lon
0,1,8919,41.397978,2.180107
1,2,8915,41.395488,2.177198
2,3,8916,41.394156,2.181331
3,4,8919,41.393317,2.181248
4,5,8907,41.391103,2.180176
...,...,...,...,...
502,515,8919,41.435207,2.194800
503,516,8919,41.435460,2.200157
504,517,8910,41.462095,2.178959
505,518,8919,41.424689,2.157049


In [11]:
gener_estacions = pd.read_csv('data/2021_01_Gener_BicingNou_ESTACIONS.csv')

gener_estacions.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,last_updated,ttl
0,1,43,43,0,3,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
1,2,17,15,2,10,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
2,3,3,3,0,22,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
3,4,15,14,1,6,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22
4,5,8,8,0,31,1.609455e+09,True,IN_SERVICE,1,1,1,1609455626,22


In [12]:
locations = gener_info[['name', 'lat', 'lon']].groupby('name').mean().reset_index()

locations.loc[1]['lat']

locations.shape

(506, 3)

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import folium

from folium.plugins import HeatMap

map = folium.Map(location=[41.3870154, 2.1700471], zoom_start=13)

for i in range(locations.shape[0]):
    folium.Marker([locations.loc[i]['lat'], locations.loc[i]['lon']], popup=locations.loc[i]['name']).add_to(map)

with open('data/CARRIL_BICI.geojson') as f:
    data = json.load(f)


folium.GeoJson(data).add_to(map)

heatmap_data = [(locations.loc[i]['lat'], locations.loc[i]['lon'], frequencies.loc[i]['frequency']) for i in range(locations.shape[0])]

HeatMap(heatmap_data).add_to(map)

map




In [42]:

map_no_markers = folium.Map(location=[41.3870154, 2.1700471], zoom_start=13)

folium.GeoJson(data).add_to(map_no_markers)

HeatMap(heatmap_data).add_to(map_no_markers)

map_no_markers

In [ ]:

import folium
import json
from folium.plugins import HeatMap

# Load the geojson file
with open('bikelanes.geojson') as f:
    data = json.load(f)

# Load the bike station data
with open('bikestations.json') as f:
    bike_data = json.load(f)

# Create a folium map centered on your city
m = folium.Map(location=[YOUR_CITY_LATITUDE, YOUR_CITY_LONGITUDE], zoom_start=12)

# Add a GeoJson layer to the map
folium.GeoJson(data).add_to(m)

# Create a list of (latitude, longitude, weight) tuples from the bike station data
heatmap_data = [(station['latitude'], station['longitude'], station['usage']) for station in bike_data]

# Add a heatmap layer to the map
HeatMap(heatmap_data).add_to(m)

# Display the map
m


In [38]:
# something like this should be done if we want to account for spain's time dts changes
import pytz
from datetime import datetime

# create a timezone object for the Spanish time zone
spain_tz = pytz.timezone('Europe/Madrid')


og = 1583503600

# create a datetime object representing a timestamp in UTC
utc_time = datetime.utcfromtimestamp(og)

# convert the UTC datetime object to the Spanish time zone
spain_time = utc_time.replace(tzinfo=pytz.utc).astimezone(spain_tz)

# convert the Spanish datetime object to a Unix timestamp
spain_timestamp = int(spain_time.timestamp())

# print the original and corrected timestamps
print("Original timestamp:", og)
print("Corrected timestamp:", spain_timestamp)

Original timestamp: 1583503600
Corrected timestamp: 1583503600
